# Import libraries

In [1]:
import pandas as pd
from pmaw import PushshiftAPI
import datetime as dt
import re
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm

# Retreive Subreddit 'wallstreetbets' with API
- Export '2021_240000_posts.csv'

In [2]:
# instantiate
api = PushshiftAPI()

# Time in 2021
before = int(dt.datetime(2021,1,31,23,59).timestamp())
after = int(dt.datetime(2021,1,1,0,0).timestamp())

# wallstreetbets
subreddit = 'wallstreetbets'

### Here I collect 20,000 posts for each month, so the total number of data is 240,000.

In [3]:
posts_df = pd.DataFrame()
for i in tqdm(range(1, 13)):
    api = PushshiftAPI()
    if i in (1,3,5,7,8,10,12):
        day = 31
    elif i == 2: 
        day = 28
    else:
        day = 30
    after = int(dt.datetime(2021,i,1,0,0).timestamp())
    before = int(dt.datetime(2021,i,day,23,59).timestamp())
    posts = api.search_submissions(subreddit = subreddit, limit=20000, before = before, after = after)
    df = pd.DataFrame(posts)
    posts_df = pd.concat([posts_df, df])

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [06:50<00:00, 410.68s/it]


In [4]:
# Example of few data
posts_df = posts_df.reset_index(drop = True)
posts_df.head(5)

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,...,media,media_embed,secure_media,secure_media_embed,author_flair_template_id,banned_by,edited,gilded,collections,distinguished
0,[],False,Vegetable-Secretary2,None,[],None,text,t2_8nndatmg,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,goofytoottt,None,[],None,text,t2_5f5hbtzt,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Vegetable-Secretary2,None,[],None,text,t2_8nndatmg,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,Vegetable-Secretary2,None,[],None,text,t2_8nndatmg,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,Untitled_DJ,None,[],None,text,t2_8kkeyj45,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Export '2021_240000_posts.csv'
posts_df.to_csv('./Datasets/2021_240000_posts.csv', index = False)

# Process the data
- Select feautures we need
- New column 'content' for all texts
- Drop data with nan in content column
- Export '2021_118990_posts_modified.csv'

In [19]:
posts_df = pd.read_csv('./Datasets/2021_240000_posts.csv')

C:\Users\USER\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,81,82,83,84,85,86,87,88,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [20]:
# Select features we need
posts_df = posts_df[['title','num_comments', 'score', 'upvote_ratio', 'selftext','created_utc']]
posts_df.created_utc = posts_df.created_utc.apply(lambda x: dt.datetime.fromtimestamp(x)) # Convert time created

# New column 'content' with combined title and selftext(posts)
posts_df['content'] = posts_df['title'] + " " + posts_df['selftext']
posts_df = posts_df.drop(['title', 'selftext'], axis=1)

# remove non-letter or space
regex = re.compile('[^a-zA-Z ]') 
posts_df['content'] = posts_df.content.apply(lambda x: regex.sub('', str(x)))

In [21]:
# Filter out
posts_df = posts_df[posts_df.content != 'nan']
posts_df = posts_df.reset_index(drop = True)
posts_df = posts_df.sort_values('created_utc')
print(posts_df.shape)
posts_df.head(5)

(118990, 5)


,num_comments,score,upvote_ratio,created_utc,content
7417,2,1,1.0,2021-01-01 00:04:00,PLEASE HELP WITH OPTIONS removed
7416,13,1,1.0,2021-01-01 00:05:22,GME Game Informer Magazine Alright so hear me ...
7415,22,1,1.0,2021-01-01 00:06:40,GME CallOption Question This question is proba...
3682,2,1,1.0,2021-01-01 00:12:38,NEED YOUR HELP GUYS removed
3681,12,1,1.0,2021-01-01 00:21:19,Heres to and tendies Whats up retards and aut...


In [22]:
# Export the modified data
posts_df.to_csv('./Datasets/2021_118990_posts_modified.csv', index = False)

# Get stock symbols existing in Nasdaq stock list & add new column with stock ticker
1. Export '2021_Symbols_list.csv'
2. Export data with new column 'ticker' corresponding to each post (if there are more than 1 stock ticker mentioned in the post, create a new row to store it) --> 'posts_with_ticker.csv'

In [23]:
posts_df = pd.read_csv('./Datasets/2021_118990_posts_modified.csv')

In [24]:
# Stock ticker list
stock = pd.read_csv("./Datasets/Nasdaq_stock.csv")
stock_symbols = list(stock.Symbol)

In [25]:
# Count the number of stock tickers
stop_words = set(stopwords.words('english'))
all_tickers = {}
ticker_col = []
for i, sentence in enumerate(posts_df['content']):
    word_tokens = word_tokenize(sentence)
    # tickers of each post
    ticker = []
    for w in word_tokens:
        if (w.lower() not in stop_words) and (w in stock_symbols): # word is not a common words and is a stock symbol
            if w not in all_tickers:
                all_tickers[w] = 1
            else:
                all_tickers[w] += 1
      
            if w not in ticker:
                ticker.append(w)
                
    if ticker == []:
        ticker_col.append('NA')
    else:
        ticker_col.append(ticker)


### 1. Export '2021_Symbols_list.csv'

In [26]:
# Sort all_tickers based on the number of stock symbol mentioned in the posts
sorted_all_tickers = sorted(all_tickers.items(), key=lambda kv: kv[1], reverse = True)
symbols_list = pd.DataFrame(sorted_all_tickers, columns = ['Symbols', 'Counts'])
symbols_list.head(5)

,Symbols,Counts
0,CLOV,3421
1,TSLA,1985
2,WISH,1923
3,AMD,1144
4,SDC,1065


In [27]:
# Export symbols_list
symbols_list.to_csv('./Datasets/2021_Symbols_list.csv', index = False)

### 2. Export 'posts_with_ticker.csv'

In [28]:
posts_df['ticker'] = ticker_col  # add new column 'ticker'
posts_df = posts_df[posts_df.ticker != 'NA'] # remove data with no tickers
posts_df = posts_df.reset_index(drop=True)

In [29]:
# extra dataframe is to store the added data with the second, third, ..., tickers
extra = pd.DataFrame()
for i in range(len(posts_df)):
    if len(posts_df.ticker[i]) > 1:
        for j in range(1, len(posts_df.ticker[i])):
            x = posts_df.iloc[i].copy()
            x.ticker = posts_df.ticker[i][j]
            extra = extra.append(x)
    
    # modify the first mentioned ticker in original posts_df after each iteration
    posts_df.ticker[i] = posts_df.ticker[i][0]
    

<ipython-input-29-3f50416766ca>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_df.ticker[i] = posts_df.ticker[i][0]


In [30]:
# Combine original and extra dataframe
posts_df_ticker = pd.concat([posts_df, extra]).sort_values('created_utc').reset_index(drop=True)
print(posts_df_ticker.shape)
posts_df_ticker.head(5)

(37991, 6)


,num_comments,score,upvote_ratio,created_utc,content,ticker
0,0.0,1.0,1.0,2021-01-01 00:54:48,Sell GOOG and buy VUG and FDN removed,GOOG
1,0.0,1.0,1.0,2021-01-01 03:30:18,Lets hope that during the new year retarded po...,TSLA
2,6.0,1.0,1.0,2021-01-01 05:19:50,sales BBY down from ath ZM down from ath Th...,ZM
3,52.0,1.0,1.0,2021-01-01 07:03:43,Is AAPL looking like a YOLO buy FACTS is xmasn...,AAPL
4,7.0,1.0,1.0,2021-01-01 08:29:45,Stop wasting your money on internet awards and...,LIFE


In [31]:
# Export 'posts_with_ticker.csv'
posts_df_ticker.to_csv('./Datasets/posts_with_ticker.csv', index = False)